In [ ]:
from google.colab import drive
drive.mount('/content/drive')


# ***ANN_Project Training File***
# Steps for data cleaning and preprocesing
1)Remove the unneccessary column/feature
<br>
2)Apply lable encoding for for binary(male,female)
<br> Onehot encoding for multi_catagerical columns(spain,france,germany)
<br>
3) save multi_catagerical columns as seperate unique columns in seperate dataset <br>
4)Save the colunms i.e encoder and scaler in pickle file formate pkl <br>
5)Split Data into Input (X) and Output (y)
<br>
6)Split into Training and Testing Sets <br>
7)Standardize the Features means scal the feature i.e <br>  reduce it mean to zero and standard devition to 1 <br>
8) Build ANN Model <br>
9)Compile ANN Model <br>
10)Setup Callbacks i.e tensorboard_callback,EarlyStopping(if model stop improvement) <br>
11)Train the Model  <br>
12)Save Model as h5 formate <br>
13)Load TensorBoard for visual representation


In [ ]:
import pandas as pd

# Example path – adjust it based on where you uploaded
file_path = '/content/drive/MyDrive/ANN_Project/Churn_Modelling.csv'

# Load the file
df = pd.read_csv(file_path)

# Show the first few rows
df.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
#  1) Drop irrelevent columns

df=df.drop(['RowNumber','CustomerId','Surname'],axis=1 )


In [ ]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
# 2) Seperate catogerical columns using label_encoder(binary)
from sklearn.preprocessing import LabelEncoder,StandardScaler
label_encoder_gender=LabelEncoder()
df['Gender']=label_encoder_gender.fit_transform(df['Gender'])

In [ ]:
# Seperate catogerical columns using onehot_encoding(multi_feature)

from sklearn.preprocessing import OneHotEncoder
one_hot_encoder=OneHotEncoder()
geo_encoder=one_hot_encoder.fit_transform(df[['Geography']]) ## # every columns have assign uniqu values that is 1*3 matrix by using fit_transform
#  here   (df[['Geography']]) we use double bricket b/c it return 2D array


In [ ]:
# 3) Geography columns change to unique columns names as france,spain,germany
one_hot_encoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [ ]:
#  it assign 1*3 matrix sparse array to newly columns
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=one_hot_encoder.get_feature_names_out(['Geography']))
geo_encoded_df  ## this is my new data frame with three columns coming from geography

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [ ]:
# now combine onehot encode colunmns with original data i.e
#  assign that value in orignal dataset insetead of text
df=pd.concat([df.drop('Geography',axis=1),geo_encoded_df],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [ ]:
# 4) saving encoder and scaler as pkl
import pickle
with open('label_encoder_gender.pkl','wb') as file:
  pickle.dump(label_encoder_gender,file)
with open('one_hot_encoder.pkl','wb') as file:
  pickle.dump(one_hot_encoder,file)

In [ ]:
#   5) Divide dataset as dependent and independent feature


from sklearn.model_selection import train_test_split
x=df.drop('Exited',axis=1)
y=df['Exited']

#   split the dataset as training and testing
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

# #   Scale these feature
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)



In [ ]:
with open('scaler.pkl','wb') as file:
  pickle.dump(scaler,file)

# **ANN** **implementation**

In [ ]:
# main library
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
import datetime


In [ ]:
input_shape=(x_train.shape[1],)
input_shape

(12,)

In [ ]:
# 8) Build ANN Model
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)), # HL1
    Dense(32,activation='relu'),  #HL2
    Dense(1,activation='sigmoid')   #  output layer

])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)  # initialize opt

In [ ]:
# 9)  compile the model  means assign or configure d/f perameter to model which we created
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
#make folder
log_dir = "logs/fit/static_run"
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [ ]:
# 10)  Setup early stoping i.e if model not perform improvement then stop
early_stoping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [ ]:
# 11)  Train the model
history=model.fit(
    x_train,y_train,validation_data=(x_test,y_test),epochs=100,
      callbacks=[tensorflow_callback,early_stoping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8124 - loss: 0.4406 - val_accuracy: 0.8555 - val_loss: 0.3591
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8551 - loss: 0.3558 - val_accuracy: 0.8580 - val_loss: 0.3503
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8607 - loss: 0.3406 - val_accuracy: 0.8605 - val_loss: 0.3489
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8597 - loss: 0.3499 - val_accuracy: 0.8635 - val_loss: 0.3565
Epoch 5/100
 48/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8551 - loss: 0.3335

In [ ]:
model.save('/content/drive/MyDrive/ANN_Project/ANN_P1/model.h5')


In [ ]:
# 13) Load tensorboard for visual representation

%reload_ext tensorboard
%tensorboard --logdir logs/fit/static_run


<IPython.core.display.Javascript object>

In [ ]:
# from google.colab import files
# files.download('model.pkl')
# files.download('label_encoder_gender.pkl')
# files.download('one_hot_encoder.pkl')
# files.download('scaler.pkl')
